In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import warnings
from review import review_parsing
from db import db_insert
import random
warnings.filterwarnings('ignore')

In [18]:
# 영화 dataframe
# 페이지 접속
driver = webdriver.Chrome()
driver.get("https://m.kinolights.com/discover/explore") # 키노라이트 탐색를 여는 녀석
time.sleep(0.3) #1초 웨이팅 없으면 리젝당할수도...

# 메뉴얼 필터링

In [10]:
import json


# 영화정보 크롤링 function
def crawling():
    req = driver.page_source
    soup = BeautifulSoup(req, 'html.parser')
    links = driver.find_elements_by_xpath('//*[@id="contents"]/section[1]/div/div/ul/li[1]/div/a')
    link_len = len(driver.find_elements_by_xpath(f'//*[@id="contents"]/section[2]/div/div/ul/li'))
    ottLinks = []
    if link_len == 0:
        link_len = len(driver.find_elements_by_xpath(f'//*[@id="contents"]/section[1]/div/div/ul/li'))
        for i in range(link_len):
            link = driver.find_elements_by_xpath(f'//*[@id="contents"]/section[1]/div/div/ul/li[{i+1}]/div/a')[0]
            ottLinks.append({"Ott":f'{link.text.split(" ")[0]}',"OttLink":link.get_attribute('href')})
            # LINKS += "{name:"+f'"{link.text.split(" ")[0]}"'+":'"+link.get_attribute('href')+"'},"
    else:
        link_len = len(driver.find_elements_by_xpath(f'//*[@id="contents"]/section[2]/div/div/ul/li'))
        for i in range(link_len):
            link = driver.find_elements_by_xpath(f'//*[@id="contents"]/section[2]/div/div/ul/li[{i+1}]/div/a')[0]
            ottLinks.append({"Name":f'{link.text.split(" ")[0]}',"OttLink":link.get_attribute('href')})
    casts = []
    try:
        names = soup.select('.photo > img ')
        photos = soup.select('.photo > img ')
        director = {"Director":f'{str(names[0]["alt"])}', "ProfilImg":f'{photos[0]["src"]}' }
        for i in range(len(photos)-1):
            if i < len(photos)-2:
                casts.append({"Actor":f'{names[i+1]["alt"]}',"ProfilImg":f'{photos[i+1]["src"]}'})
            else:
                casts.append({"Actor":f'{names[i+1]["alt"]}',"ProfilImg":f'{photos[i+1]["src"]}'})
    except IndexError:
        casts = None
        director = None
    try:
        trailer = driver.find_element_by_tag_name('iframe').get_attribute('src')
        trailer_split = trailer.split('embed/')[1]
        trailer_key = trailer_split.split('.kino')[0]
    except:
        trailer = None
        trailer_key = "1nH5dwSPamg"
    try:
        info = driver.find_element_by_css_selector("#synopsis > article:nth-child(1) > p").text
    except:
        info = None

    dict = { 'TitleKr' : driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[5]/div[1]/h3').text,
            'TitleEn' : driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/h4")[0].text,
            'Poster' : driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[5]/div[2]/img').get_attribute('src'),
            'Genre' : driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/p/span[1]")[0].text[:-2],
            'Year' : driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/p/span[2]")[0].text,
            'Kino' : None if driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[1]/div")[0].text[:-1] =='- ' else float(driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[1]/div")[0].text[:-1]),
            'Tomato' : None if driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[2]/span[1]/div")[0].text[:-3]=='- '
            else float(driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[2]/span[1]/div")[0].text[:-3]),
            'Imdb' :None if driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[2]/span[2]/div")[0].text[:-2]=='-'
             else float(driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[2]/span[2]/div")[0].text[:-2]),
            'Star' : None  if driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[2]/div/div")[0].text[:]=='-'
            else float(driver.find_elements_by_xpath("/html/body/div/div/div/main/div[2]/div[5]/div[1]/div/div/div[2]/div/div")[0].text[:]),
            'OttLinks' : ottLinks,
            'Director' :  json.dumps(director, ensure_ascii=False),
            'Casts' : casts,
            'Trailer' : trailer,
            'TrailerKey' : trailer_key,
            'Info' : info ,
            'KinoId': int(driver.current_url.split("/")[-1]) }
    return dict

In [ ]:
crawling()

# 스크롤다운

In [11]:
time.sleep(3)
#  스크롤다운
SCROLL_PAUSE_SEC = 1
# 스크롤 높이 가져옴
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # try:
    # 끝까지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print("스크롤 완료!")
        break
    last_height = new_height
    # except:
    #     print("스크롤 완료")
    #     break
#iframe ? 스크롤완료시 오류 없음

스크롤 완료!


# get link

In [12]:
from selenium.webdriver import ActionChains
cnt = 0
links = []
# 맄크 가져오기
while True:
    try:
        cnt += 1
        url = f'//*[@id="listArea"]/div[3]/div[2]/div[{cnt}]/a'
        url = f'//*[@id="contents"]/main/div[1]/div[3]/div[2]/div[{cnt}]/a'
        link = driver.find_element_by_xpath(url).get_attribute('href')
        links.append(link)
        # driver.get(link)
        # time.sleep(0.3)
        # print(crawling())
        # driver.back()
        # time.sleep(0.3)
        # if cnt==3:
        #     break
    except:
        break
# for link in links:
#     #print(link)
print(f'{len(links)} 가져옴!')

697 가져옴!


# main 함수

In [19]:
def main(inputlinks):
    i = 0
    s = 0
    e = 0
    err_links =[]
    while i<len(links):
        try:
            time.sleep(random.randrange(1,7))
            driver.get(inputlinks[i])
            time.sleep(random.randrange(1,3))
            # 컬럼설정 부분 필요 없음 딕셔너리에서 바로 df return
            m_df =  pd.DataFrame(columns = [ 'TitleKr', 'TitleEn', 'Poster','Genre', 'Year', 'Kino', 'Tomato', 'Imdb', 'Star','OttLinks', 'Info', 'Director', 'Casts', 'Trailer','TrailerKey', 'KinoId'])
            m_df = m_df.append(crawling(), ignore_index=True)
            m_df['OttLinks'] = m_df['OttLinks'].astype('str')
            m_df['Casts'] = m_df['Casts'].astype('str')
            db_insert(m_df,'MovieInfo')
            kino_id = inputlinks[i].split("/")[-1]
            try:
                db_insert(review_parsing(int(kino_id)),'MovieReview')
            except:
                print(f'get review err {kino_id}')
            i += 1
            s += 1
            print(f'{i}/{len(inputlinks)} 완료!')
        except IndexError as IE:
            print(f'{links[i]} err :: {IE}')
            err_links.append(links[i])
            i += 1
            e += 1
            continue
        except all as msg:
            print(f'{i}번 err: {msg}')
            i += 1
            e += 1
            err_links.append(links[i])
            continue
    print(f'크롤링 완료~~~~~~~~~!!!!!!!!!!!!!!!{s}/{e}')
    return err_links

In [ ]:
links[0]

In [ ]:
157
156

# RUN

In [20]:
err_links = main(links)
print(f'{len(err_links)} 개 에러 링크 취득!')
err_links

MovieInfo :: 205 insert success
MovieReview :: 645 insert success
201/697 완료!
MovieInfo :: 205 insert success
MovieReview :: 645 insert success
202/697 완료!
MovieInfo :: 205 insert success
MovieReview :: 645 insert success
203/697 완료!
MovieInfo :: 205 insert success
MovieReview :: 645 insert success
204/697 완료!
MovieInfo :: 205 insert success
MovieReview :: 645 insert success
205/697 완료!
MovieInfo :: 205 insert success
MovieReview :: 645 insert success
206/697 완료!
MovieInfo :: 205 insert success
MovieReview :: 646 insert success
207/697 완료!
MovieInfo :: 206 insert success
MovieReview :: 652 insert success
208/697 완료!
MovieInfo :: 207 insert success
MovieReview :: 657 insert success
209/697 완료!
MovieInfo :: 208 insert success
MovieReview :: 663 insert success
210/697 완료!
MovieInfo :: 209 insert success
MovieReview :: 672 insert success
211/697 완료!
MovieInfo :: 210 insert success
MovieReview :: 679 insert success
212/697 완료!
MovieInfo :: 211 insert success
MovieReview :: 688 insert succes

[]

In [ ]:
def err_check(err_link):
    while True:
            time.sleep(random.randrange(1,7))
            driver.get(err_link)
            time.sleep(random.randrange(1,3))
            m_df =  pd.DataFrame(columns = [ 'TITLE_KR', 'TITLE_EN', 'POSTER','GENRE', 'YEAR', 'KINO', 'TOMATO', 'IMDB', 'STAR','LINK', 'INFO', 'DIRECTOR', 'CAST', 'TRAILER', 'KINO_ID'])
            m_df = m_df.append(crawling(), ignore_index=True)
            kino_id = err_link.split("/")[-1]
            db_insert(m_df,'movie_info')
            db_insert(review_parsing(int(kino_id)),'movie_review')
            print(f'{i}번 완료!')

In [ ]:
err_check(err_links[0])